In [2]:
import os
import json
from collections import namedtuple

In [3]:
target_conditions = set(['FRDA', 'DM1', 'HD', 'FXS'])

## Define paths

In [4]:
inputs = '../../input/'
scratch = '../../scratch/'
tools = '../../tools/'
reference = '../../../../common/refs/grch37/human_g1k_v37.fasta'

## Load coriell info

In [5]:
Phenotype = namedtuple('Phenotype', 'id condition status')

phenotypes = ! cat {inputs}/coriell_phenotype_table.tsv
phenotypes = [Phenotype(id=rec.split()[0], condition=rec.split()[1], status=rec.split()[2])
              for rec in phenotypes]

phenotypes = [rec for rec in phenotypes
              if rec.condition in target_conditions and rec.status != 'Normal']

print(len(phenotypes))
phenotypes

91


[Phenotype(id='NA06075', condition='DM1', status='Expansion'),
 Phenotype(id='NA04567', condition='DM1', status='Expansion'),
 Phenotype(id='NA05164', condition='DM1', status='Expansion'),
 Phenotype(id='NA04648', condition='DM1', status='Expansion'),
 Phenotype(id='NA05152', condition='DM1', status='Expansion'),
 Phenotype(id='NA23378', condition='DM1', status='Expansion'),
 Phenotype(id='NA23374', condition='DM1', status='Expansion'),
 Phenotype(id='NA23300', condition='DM1', status='Expansion'),
 Phenotype(id='NA03986', condition='DM1', status='Expansion'),
 Phenotype(id='NA03989', condition='DM1', status='Expansion'),
 Phenotype(id='NA03990', condition='DM1', status='Expansion'),
 Phenotype(id='NA03696', condition='DM1', status='Expansion'),
 Phenotype(id='NA03759', condition='DM1', status='Expansion'),
 Phenotype(id='NA04034', condition='DM1', status='Expansion'),
 Phenotype(id='NA03697', condition='DM1', status='Expansion'),
 Phenotype(id='NA03132', condition='DM1', status='Expan

## Generate manifest for a given repeat

In [31]:
def get_control_records():
    records = []
    dir_path = os.path.join(inputs, 'Diversity/')
    for fname in os.listdir(dir_path):
        sample = fname.replace('.str_profile.json', '')
        file_path = os.path.abspath(os.path.join(dir_path, fname))
        records.append((sample, 'control', file_path))
    return records

In [33]:
len(get_control_records())

150

In [48]:
def get_case_records(condition):
    records = []
    target_samples = [pheno.id for pheno in phenotypes if pheno.condition == condition]
    dir_path = os.path.join(inputs, 'RepeatExpansions')
    for fname in os.listdir(dir_path):
        file_path = os.path.abspath(os.path.join(dir_path, fname))
        sample = fname.replace('.str_profile.json', '')
        if sample in target_samples:
            records.append((sample, 'case', file_path))
    return records

In [56]:
for condition in target_conditions:
    records = get_case_records(condition)
    records.extend(get_control_records())
    
    lines = ['\t'.join(line) for line in records]
    with open(os.path.join(scratch, condition, 'manifest.tsv'), 'w') as manifest_file:
        print('\n'.join(lines), file=manifest_file)

## Perform case-control analysis

In [10]:
%%bash -s "$tools" "$scratch" "$reference"

tools=$1
scratch=$2
reference=$3

#for condition in FRDA DM1 HD FXS
#do
#  $tools/ExpansionHunterDenovo/build/ExpansionHunterDenovo merge \
#    --reference $reference \
#    --manifest $scratch/$condition/manifest.tsv \
#    --output-prefix $scratch/$condition/dataset \
#    --min-unit-len 2 \
#    --max-unit-len 15
#done

for condition in FRDA DM1 HD FXS
do
  $tools/ExpansionHunterDenovo/scripts/casecontrol.py locus \
    --manifest $scratch/$condition/manifest.tsv \
    --min-inrepeat-reads 5 \
    --test-params normal \
    --multisample-profile $scratch/$condition/dataset.multisample_profile.json \
    --output $scratch/$condition/locus.case-control.tsv
done


for condition in FRDA DM1 HD FXS
do
  $tools/ExpansionHunterDenovo/scripts/casecontrol.py motif \
    --manifest $scratch/$condition/manifest.tsv \
    --multisample-profile $scratch/$condition/dataset.multisample_profile.json \
    --output $scratch/$condition/motif.case-control.tsv \
    --min-inrepeat-read-pairs 5 \
    --test-params normal
done


2019-11-06 14:35:38,913: Loaded 81499 regions
2019-11-06 14:35:38,913: Normalizing counts
2019-11-06 14:35:39,113: Filtering counts
2019-11-06 14:35:39,207: 2263 regions left after filtering
2019-11-06 14:35:39,207: Comparing counts
2019-11-06 14:35:39,805: Correcting p-values
2019-11-06 14:35:40,038: Done
2019-11-06 14:35:40,703: Loaded 80460 regions
2019-11-06 14:35:40,703: Normalizing counts
2019-11-06 14:35:40,890: Filtering counts
2019-11-06 14:35:40,982: 2277 regions left after filtering
2019-11-06 14:35:40,982: Comparing counts
2019-11-06 14:35:41,471: Correcting p-values
2019-11-06 14:35:41,701: Done
2019-11-06 14:35:42,354: Loaded 78810 regions
2019-11-06 14:35:42,354: Normalizing counts
2019-11-06 14:35:42,537: Filtering counts
2019-11-06 14:35:42,628: 2253 regions left after filtering
2019-11-06 14:35:42,628: Comparing counts
2019-11-06 14:35:43,112: Correcting p-values
2019-11-06 14:35:43,334: Done
2019-11-06 14:35:44,041: Loaded 85286 regions
2019-11-06 14:35:44,041: Norma